In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import urllib.request
import requests

In [2]:
url = "https://media.naver.com/press/023/ranking?type=comment&date=20230912"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'lxml')

In [3]:
newslist = soup.select(".press_ranking_list")
len(newslist)

2

In [4]:
newslist[0]

<ul class="press_ranking_list">
<li class="as_thumb">
<a class="_es_pc_link" data-pc-link="" href="https://n.news.naver.com/article/023/0003787280?ntype=RANKING" onclick="nclk(event, 'rnk.cmtnws','','')">
<em class="list_ranking_num">1</em>
<div class="list_content">
<strong class="list_title">이재명, 단식 전날 오염수 집회뒤 횟집식사…與 “몰래 잡순 ‘날것’ 입에 맞았나”</strong>
<span class="list_comment">
<span class="blind">댓글</span>
                                                1,483
                                            
                                        
                                    </span>
</div>
<div class="list_img">
<img height="100%" onerror="showNoImage(this)" src="https://mimgnews.pstatic.net/image/origin/023/2023/09/12/3787280.jpg?type=nf148_148" width="100%"/>
</div>
</a>
</li>
<li class="as_thumb">
<a class="_es_pc_link" data-pc-link="" href="https://n.news.naver.com/article/023/0003787311?ntype=RANKING" onclick="nclk(event, 'rnk.cmtnws','','')">
<em class="list_ranking_num">2</em>

## 댓글 많은 뉴스 가져오기
- 랭킹, 제목, 링크만

In [5]:
newsData = []

for news in newslist:
    # 댓글 많은 뉴스 상위 20개 가져옴
    lis = news.findAll("li")
    for li in lis:
        # 뉴스랭킹
        news_ranking = li.select_one(".list_ranking_num").text
        # 뉴스링크와 제목
        list_title = li.select_one(".list_title")
        news_title = list_title.text
        news_link = li.select_one("._es_pc_link").get("href")
        #print("랭킹: ", news_ranking)
        #print("제목: ", news_title)
        #print("링크: ", news_link)
        
        # 저장
        newsData.append({
            'ranking': news_ranking,
            'title': news_title,
            'link': news_link,
        })

In [6]:
newsData

[{'ranking': '1',
  'title': '이재명, 단식 전날 오염수 집회뒤 횟집식사…與 “몰래 잡순 ‘날것’ 입에 맞았나”',
  'link': 'https://n.news.naver.com/article/023/0003787280?ntype=RANKING'},
 {'ranking': '2',
  'title': '[속보] 이종섭 국방장관 사의 표명... 尹대통령 수리할 듯',
  'link': 'https://n.news.naver.com/article/023/0003787311?ntype=RANKING'},
 {'ranking': '3',
  'title': 'MBC 3노조 “대선 전날밤 ‘PD수첩’은 미디어 이용 부정선거”… 방송 어땠길래',
  'link': 'https://n.news.naver.com/article/023/0003787302?ntype=RANKING'},
 {'ranking': '4',
  'title': '이화영에 떠넘긴 이재명 “나 몰래 독단적으로 대북사업 추진”',
  'link': 'https://n.news.naver.com/article/023/0003787256?ntype=RANKING'},
 {'ranking': '5',
  'title': '강남서 람보르기니 타고 “칼침 맞아봤냐”… 30대 남성, 마약 3종 양성',
  'link': 'https://n.news.naver.com/article/023/0003787291?ntype=RANKING'},
 {'ranking': '6',
  'title': 'KBS이사회, 김의철 사장 해임 제청안 의결...尹 재가하면 확정',
  'link': 'https://n.news.naver.com/article/023/0003787317?ntype=RANKING'},
 {'ranking': '7',
  'title': '이재명, 대북송금 2차 검찰 출두... “역사가 심판할 것”',
  'link': 'https://n.news.naver.com/article/023/

## 각 뉴스의 내용, 작성일자, 카테고리 가져오기

In [7]:
for news in newsData:
    news_url = news['link']
    res = requests.get(news_url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    # 작성시간
    news_time = soup.select_one(".media_end_head_info_datestamp").select_one(".media_end_head_info_datestamp_time").get("data-date-time")
    # 뉴스 내용
    news_content = soup.select_one("#newsct_article").text.replace("\n","").replace("\t","")
    # 섹션 분류
    news_category = soup.select_one(".media_end_categorize").select_one(".media_end_categorize_item").text
    news['time'] = news_time
    news['contents'] = news_content
    news['category'] = news_category

In [8]:
newsData

[{'ranking': '1',
  'title': '이재명, 단식 전날 오염수 집회뒤 횟집식사…與 “몰래 잡순 ‘날것’ 입에 맞았나”',
  'link': 'https://n.news.naver.com/article/023/0003787280?ntype=RANKING',
  'time': '2023-09-12 08:47:07',
  'contents': '이재명 더불어민주당 대표가 지난달 30일 목포의 한 횟집에 방문해 남긴 방명록. /페이스북이재명 더불어민주당 대표가 지난달 30일 일본 후쿠시마 제1원전 오염수 방류 규탄대회를 마친 뒤 자당 의원들과 함께 횟집에서 식사를 했다는 사실이 뒤늦게 알려졌다. 오염수 방류 일주일째 되는 날이자, 이 대표가 단식을 시작하기 하루 전날이었다. 국민의힘은 “국민 몰래 잡순 ‘날것’들은 입에 맞았느냐”고 비판했다.김민수 국민의힘 대변인은 11일 논평을 내고 “자기기인(自欺欺人). 이 대표는 자신도 믿지 않는 행동으로 국민을 속였다”고 했다.김 대변인은 “이재명의 ‘후쿠시마 오염수’는 자신의 범죄 혐의에 대한 방탄용이자 묻지마식 ‘윤석열 정권퇴진운동’을 위한 수단일 뿐이었다는 것이 드러났다”고 했다.김 대변인은 “단식 하루 전, 몸에 좋은 해산물로 영양소는 충분히 채우셨나”라며 “‘날 것’을 이리 좋아하시니, 단식 또한 날로 먹고 있는 것은 아니냐는 우스갯소리가 나온다”고 했다. 이어 “자신들도 믿지 않은 후쿠시마 오염수의 위험성을 국민들께 유포하고 거짓 선동하며 국론을 분열시키는 ‘죄’, 어찌 가볍다고 하겠느냐”라고 했다.이 대표가 11일 국회 본청 앞 단식 농성 천막에 누워있다./연합뉴스앞서 이날 한 언론보도에 따르면 이 대표는 자당 소속 의원들과 함께 목포시 소재의 한 횟집에서 식사를 했다. 횟집 사장이 소셜미디어에 이 대표를 비롯한 민주당 의원들의 기념사진을 올리면서 뒤늦게 이 사실이 알려졌다. 이 대표는 식사를 마친 뒤 남긴 방명록에 ‘함께 사는 세상. OO횟집, 참 맛있게 잘 먹었습니다’라는 글귀를 남겼다.이 대

## 데이터프레임으로 만들기

In [9]:
joseon = pd.DataFrame(newsData)
joseon

,ranking,title,link,time,contents,category
0,1,"이재명, 단식 전날 오염수 집회뒤 횟집식사…與 “몰래 잡순 ‘날것’ 입에 맞았나”",https://n.news.naver.com/article/023/000378728...,2023-09-12 08:47:07,이재명 더불어민주당 대표가 지난달 30일 목포의 한 횟집에 방문해 남긴 방명록. /...,정치
1,2,[속보] 이종섭 국방장관 사의 표명... 尹대통령 수리할 듯,https://n.news.naver.com/article/023/000378731...,2023-09-12 11:17:14,"尹대통령, 13일 신임 국방장관 후보 지명할 듯후임 신원식 유력이종섭 국방부 장관이...",정치
2,3,MBC 3노조 “대선 전날밤 ‘PD수첩’은 미디어 이용 부정선거”… 방송 어땠길래,https://n.news.naver.com/article/023/000378730...,2023-09-12 11:04:12,김만배 짜깁기 녹음 그대로 1분간 들려주며 해설 자막엔 “尹이 어떻게 관여했는지 ...,생활
3,4,이화영에 떠넘긴 이재명 “나 몰래 독단적으로 대북사업 추진”,https://n.news.naver.com/article/023/000378725...,2023-09-12 04:08:08,쌍방울 사건 검찰 조사서 혐의 부인더불어민주당 이재명 대표가 지난 9일 오후 경기도...,사회
4,5,"강남서 람보르기니 타고 “칼침 맞아봤냐”… 30대 남성, 마약 3종 양성",https://n.news.naver.com/article/023/000378729...,2023-09-12 10:04:08,유튜브 '카라큘라 탐정사무소' 갈무리./뉴스1서울 강남에서 람보르기니 차량을 주차하...,사회
5,6,"KBS이사회, 김의철 사장 해임 제청안 의결...尹 재가하면 확정",https://n.news.naver.com/article/023/000378731...,2023-09-12 11:30:13,12일 오전 서울 여의도 KBS에 김인철 사장 퇴출 관련 포스터가 부착돼 있다./뉴...,생활
6,7,"이재명, 대북송금 2차 검찰 출두... “역사가 심판할 것”",https://n.news.naver.com/article/023/000378733...,2023-09-12 13:24:09,이재명 민주당 대표가 12일 오후 1시 23분 ‘쌍방울 불법 대북 송금 사건’으로 ...,사회
7,8,“아이 손이 친구 뺨에 맞았다” 대전 사망 교사 관련 학부모 해명에 비난 쏟아져,https://n.news.naver.com/article/023/000378731...,2023-09-12 11:15:15,"학부모 “면담서 ‘인민재판식’ 훈육 말라, 아이에 사과 요청”악성민원에 시달리다 극...",사회
8,9,민주당 의총서 “검찰 아가리에 이재명 내줄 수 없어”,https://n.news.naver.com/article/023/000378732...,2023-09-12 12:26:12,더불어민주당은 12일 이재명 대표의 수원지검 출석을 앞두고 의원총회를 열고 “잔혹하...,정치
9,10,"울산시장 선거 개입, 이성윤은 기소 늦추고 김명수 법원은 재판 미뤄",https://n.news.naver.com/article/023/000378725...,2023-09-12 04:07:05,"‘친문’ 李, 기소 결재 요청 3번 무시이성윤 법무연수원 연구위원(왼쪽), 김명수 ...",사회


## 정치 섹션만 남기기

In [10]:
final_js = joseon[joseon['category'] == '정치']
final_js.reset_index(inplace=True)

In [11]:
final_js

,index,ranking,title,link,time,contents,category
0,0,1,"이재명, 단식 전날 오염수 집회뒤 횟집식사…與 “몰래 잡순 ‘날것’ 입에 맞았나”",https://n.news.naver.com/article/023/000378728...,2023-09-12 08:47:07,이재명 더불어민주당 대표가 지난달 30일 목포의 한 횟집에 방문해 남긴 방명록. /...,정치
1,1,2,[속보] 이종섭 국방장관 사의 표명... 尹대통령 수리할 듯,https://n.news.naver.com/article/023/000378731...,2023-09-12 11:17:14,"尹대통령, 13일 신임 국방장관 후보 지명할 듯후임 신원식 유력이종섭 국방부 장관이...",정치
2,8,9,민주당 의총서 “검찰 아가리에 이재명 내줄 수 없어”,https://n.news.naver.com/article/023/000378732...,2023-09-12 12:26:12,더불어민주당은 12일 이재명 대표의 수원지검 출석을 앞두고 의원총회를 열고 “잔혹하...,정치
3,14,15,"조응천 “이재명 찾아가 눈물흘린 박지현, 초현실적이고 그로테스크”",https://n.news.naver.com/article/023/000378729...,2023-09-12 10:47:04,박지현 전 더불어민주당 비상대책위원장이 단식 중인 이재명 민주당 대표를 찾아 “회복...,정치
4,15,16,국회에 6·25전범 모택동 흉상 전시... 받침대엔 ‘구세주’,https://n.news.naver.com/article/023/000378717...,2023-09-11 22:37:06,2023 한중 도예전 전시품에 포함 민주당 김민철 “부적절 판단... 철거할 것”...,정치
5,17,18,첫날 2052명 “이승만 기념관에 힘 보탭니다”...배우 이영애도 동참,https://n.news.naver.com/article/023/000378725...,2023-09-12 03:47:02,전국 각지 평범한 시민들이 나서이승만대통령기념관건립추진위원회가 범국민 모금 운동을 ...,정치
6,19,20,양향자 “당장 단식 중인 이재명 만나라” 김기현 “그런 방식은 어려워”,https://n.news.naver.com/article/023/000378731...,2023-09-12 11:23:04,양향자 한국의희망 공동대표가 12일 김기현 국민의힘 대표를 예방했다. 한국의희망은 ...,정치


# 최종 코드
- 3월부터 8월까지 기사 크롤링하기

In [2]:
import os
from bs4 import BeautifulSoup
import time
import json
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import urllib.request
import requests

In [3]:
start_date = pd.to_datetime('2023-04-01')
end_date = pd.to_datetime('2023-08-31')
dates = pd.date_range(start_date, end_date, freq='D')
dates = list(dates.astype(str))
date = [i.replace('-', '') for i in dates]
date

['20230401',
 '20230402',
 '20230403',
 '20230404',
 '20230405',
 '20230406',
 '20230407',
 '20230408',
 '20230409',
 '20230410',
 '20230411',
 '20230412',
 '20230413',
 '20230414',
 '20230415',
 '20230416',
 '20230417',
 '20230418',
 '20230419',
 '20230420',
 '20230421',
 '20230422',
 '20230423',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230429',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230528',
 '20230529',
 '20230530',
 '20230531',
 '20230601',
 '20230602',
 '20230603',
 '20230604',
 '20230605',
 '20230606',
 '20230607',
 '20230608',
 '20230609',
 '20230610',
 '20230611',
 '20230612',
 '20230613',
 '20230614',
 '20230615',
 '20230616',

In [4]:
from tqdm import tqdm
from lxml import html

newsData = []

for d in tqdm(date):
    url = "https://media.naver.com/press/023/ranking?type=comment&date="+d
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    tree = html.fromstring(res.text)

    
    # 뉴스 가져오기
    newslist = soup.select(".press_ranking_list")
    for news in newslist:
        # 댓글 많은 뉴스 상위 20개 가져옴
        lis = news.findAll("li")
        for li in lis:
            # 뉴스랭킹
            news_ranking = li.select_one(".list_ranking_num").text
            # 뉴스제목
            list_title = li.select_one(".list_title")
            news_title = list_title.text
            # 뉴스링크
            news_link = li.select_one("._es_pc_link").get("href")

            newsData.append({
                    'ranking': news_ranking,
                    'title': news_title,
                    'link': news_link,
                })
        
        for news in newsData:
            news_url = news['link']
            res = requests.get(news_url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            
            
            # 뉴스 날짜
            try:
                news_time = soup.select_one(".media_end_head_info_datestamp").select_one("span.media_end_head_info_datestamp_time")['data-date-time']
            except:    
                news_time = tree.xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span')
            
            
            # 뉴스 내용
            try:
                news_content = soup.select_one("#newsct_article").text.replace("\n","").replace("\t","")
            except:
                news_content_list = tree.xpath('//*[@id="contents"]')
                news_content  = ''.join(news_content_list).replace("\n", "").replace("\t", "")

            
            # 섹션 분류
            try:
                news_category = soup.select_one(".media_end_categorize").select_one(".media_end_categorize_item").text
            except:
                news_category = tree.xpath('//*[@id="contents"]/div[5]/a/em')

                
            
            news['time'] = news_time
            news['contents'] = news_content
            news['category'] = news_category

 25%|██▍       | 38/153 [1:15:16<3:47:49, 118.87s/it]


KeyboardInterrupt: 

In [5]:
# 데이터프레임으로 만들기
df = pd.DataFrame(newsData)

In [6]:
df

,ranking,title,link,time,contents,category
0,1,"故 박원순 묘 이장, 묘소 직원 출근 전 새벽에 끝내",https://n.news.naver.com/article/023/000375537...,2023-04-01 10:45:01,1일 오후 경기도 남양주시 모란공원 민주열사묘역에서 열린 고(故) 박원순 전 서울...,정치
1,2,헌법 위 민주당...대통령 임명권 줄이고 조약 통제하는 법안까지 발의,https://n.news.naver.com/article/023/000375535...,2023-04-01 03:06:39,대통령의 대법원장 임명권을 제한하는 법안을 발의해 위헌 논란을 빚은 더불어민주당이 ...,정치
2,3,“택배 못 받아” 거짓말 신고했더니…생수 240㎏ 배달시킨 고객,https://n.news.naver.com/article/023/000375538...,2023-04-01 12:50:01,2월 8일 서울 강남구 한 빌라 4층에 생수 4팩을 배송 중인 택배기사의 CCTV ...,사회
3,4,시청역 도배한 전장연 스티커… 5시간 긁었지만 절반도 못뗐다,https://n.news.naver.com/article/023/000375533...,2023-04-01 03:06:02,"미관 해치고 미끄러워 사고 우려, 지하철 미화원들이 떼려다 포기… 외주업체 5시간 ...",사회
4,5,"이해찬 “이재명은 담금질 중, 검찰은 무능한 놈들”",https://n.news.naver.com/article/023/000375538...,2023-04-01 14:08:01,"이재명 더불어민주당 대표(왼쪽부터)와 박홍근 원내대표, 이해찬 상임고문이 지난달 1...",정치
...,...,...,...,...,...,...
775,16,김남국 코인 논란에... 민주당 의원들도 “정치인 자세 아냐”,https://n.news.naver.com/article/023/000376253...,NaN,NaN,NaN
776,17,"한전, 1분기도 5조 적자... 3년간 누적 50조 이를 듯",https://n.news.naver.com/article/023/000376261...,NaN,NaN,NaN
777,18,내일 총선이면 어느당 투표? 국민의힘 32% 민주당 30% [메트릭스],https://n.news.naver.com/article/023/000376252...,NaN,NaN,NaN
778,19,與 “책방서 노동력 착취하려 한 文…이재명에 ‘열정페이 미수’ 제보”,https://n.news.naver.com/article/023/000376251...,NaN,NaN,NaN


In [7]:
# 정치 뉴스만 남기기
df = df[df['category'] == '정치']

In [8]:
df

,ranking,title,link,time,contents,category
0,1,"故 박원순 묘 이장, 묘소 직원 출근 전 새벽에 끝내",https://n.news.naver.com/article/023/000375537...,2023-04-01 10:45:01,1일 오후 경기도 남양주시 모란공원 민주열사묘역에서 열린 고(故) 박원순 전 서울...,정치
1,2,헌법 위 민주당...대통령 임명권 줄이고 조약 통제하는 법안까지 발의,https://n.news.naver.com/article/023/000375535...,2023-04-01 03:06:39,대통령의 대법원장 임명권을 제한하는 법안을 발의해 위헌 논란을 빚은 더불어민주당이 ...,정치
4,5,"이해찬 “이재명은 담금질 중, 검찰은 무능한 놈들”",https://n.news.naver.com/article/023/000375538...,2023-04-01 14:08:01,"이재명 더불어민주당 대표(왼쪽부터)와 박홍근 원내대표, 이해찬 상임고문이 지난달 1...",정치
5,6,‘100주년’ 대구 서문시장 찾은 尹... “땀 흘리는 국민이 잘 살아야”,https://n.news.naver.com/article/023/000375539...,2023-04-01 18:32:01,[대구=뉴시스] 전신 기자 = 윤석열 대통령과 부인 김건희 여사가 1일 대구 서문시...,정치
10,11,"‘경북고 레전드’와 함께... 尹, 대구서 프로야구 개막전 시구",https://n.news.naver.com/article/023/000375538...,2023-04-01 15:41:08,윤석열 대통령(오른쪽)이 1일 오후 대구 삼성라이온즈파크에서 열린 프로야구 '신한은...,정치
...,...,...,...,...,...,...
765,6,"‘내돈내투’라는 김남국, 과거엔 “변호사 시절 월 100만원도 못 벌어”",https://n.news.naver.com/article/023/000376257...,2023-05-09 14:37:01,김남국 의원. /뉴스160억원대 가상화폐(코인) 보유 논란에 휩싸인 김남국 더불어민...,정치
766,7,"전세금 전액 6억 주식, 1년 안돼 4억 수익... 김남국 간 큰 베팅 시점 보니",https://n.news.naver.com/article/023/000376264...,2023-05-09 20:25:01,"김남국, 김어준 유튜브 출연해 코인 투자 원금 해명 “전세금까지 빼서 전재산 LG...",정치
767,8,"김남국, 코인 논란 나흘 만에 사과… “책임 있는 자세 보이지 못했다”",https://n.news.naver.com/article/023/000376257...,2023-05-09 14:01:05,김남국 더불어민주당 의원./뉴스1거액의 가상화폐 투자로 논란이 된 더불어민주당 김남...,정치
768,9,"주식 판 전액 코인 샀다는 김남국, 불어난 예금 10억은 어디서",https://n.news.naver.com/article/023/000376240...,2023-05-08 21:37:01,작년 재산신고 때 예금 10억원 증가 이 돈도 출처는 ‘주식 매도’로 기재김남국 ...,정치


In [9]:
# 3월 데이터 불러와서 합치기
df3 = pd.read_csv('joseon_politics.csv', index_col=0)
final_data = pd.concat([df3, df], ignore_index=True)

In [10]:
final_data

,index,ranking,title,link,time,contents,category
0,0.0,1,“1급 역적 수박 자수하라” 李 지지자들 반란표 색출 광풍,https://n.news.naver.com/article/023/000374899...,2023-03-01 03:12:01,"李 강성 지지자들 문자 폭탄무기명투표 부정, 자백 강요이름·전화 공개 ‘살생부’까지...",정치
1,1.0,2,"친명 안민석 “이재명 사퇴, 전 당원 투표로 결정하자”",https://n.news.naver.com/article/023/000374905...,2023-03-01 11:29:01,더불어민주당 이재명(오른쪽) 대표와 안민석 의원이 지난달 23일 국회에서 열린 전국...,정치
2,3.0,4,태영호 “3·1절 독립운동가 사진에 이승만 빠져...功 인정해야”,https://n.news.naver.com/article/023/000374907...,2023-03-01 16:05:01,윤석열 대통령이 1일 서울 중구 유관순 기념관에서 열린 제104주년 3·1절 기념식...,정치
3,7.0,8,"‘비명계 살생부’ 퍼지자… 이재명 “단합에 도움 안 돼, 중단해야”",https://n.news.naver.com/article/023/000374896...,2023-02-28 23:30:01,이재명 더불민주당 대표가 23일 오전 서울 여의도 국회에서 열린 기자간담회에서 발언...,정치
4,8.0,9,"진중권 “이재명 영업사원 비유한 한동훈, 표결에 영향 줬을 것”",https://n.news.naver.com/article/023/000374909...,2023-03-01 19:54:01,"진중권 광운대 특임교수(왼쪽), 한동훈 법무장관/ 조선DB이재명 더불어민주당 대표 ...",정치
...,...,...,...,...,...,...,...
589,NaN,6,"‘내돈내투’라는 김남국, 과거엔 “변호사 시절 월 100만원도 못 벌어”",https://n.news.naver.com/article/023/000376257...,2023-05-09 14:37:01,김남국 의원. /뉴스160억원대 가상화폐(코인) 보유 논란에 휩싸인 김남국 더불어민...,정치
590,NaN,7,"전세금 전액 6억 주식, 1년 안돼 4억 수익... 김남국 간 큰 베팅 시점 보니",https://n.news.naver.com/article/023/000376264...,2023-05-09 20:25:01,"김남국, 김어준 유튜브 출연해 코인 투자 원금 해명 “전세금까지 빼서 전재산 LG...",정치
591,NaN,8,"김남국, 코인 논란 나흘 만에 사과… “책임 있는 자세 보이지 못했다”",https://n.news.naver.com/article/023/000376257...,2023-05-09 14:01:05,김남국 더불어민주당 의원./뉴스1거액의 가상화폐 투자로 논란이 된 더불어민주당 김남...,정치
592,NaN,9,"주식 판 전액 코인 샀다는 김남국, 불어난 예금 10억은 어디서",https://n.news.naver.com/article/023/000376240...,2023-05-08 21:37:01,작년 재산신고 때 예금 10억원 증가 이 돈도 출처는 ‘주식 매도’로 기재김남국 ...,정치


In [10]:
final_data.to_csv('joseon_final.csv', encoding='utf-8')